# Library tutorial

This is a walkthrough of the main `contrast` classes corresponding to motors and detectors. These are the classes that are most often written to keep up with beamline developments and experimental needs. It shows in notebook format how to create, operate on, and write your own classes representing beamline components.

## Motors

### Pure Python library

Motors represent things that the user can directly affect. Obviously actual stepper, piezo, and servo motors, but also other things that are typically set from software, like temperatures or output voltages.

The first thing to note is that contrast objects need names. Contrast (actually, the `Gadget` base class) uses these names to refer to objects in the inheritance tree. We need to supply the name explicitly, because Python objects don't have names, any number of references (variable names) can be made to point at the same object. We'll use the name (`motor`) for the `Gadget` and for the variable which refers to it.

With that said, create a dummy motor to see what they can do!

In [ ]:
from contrast.motors import DummyMotor
motor = DummyMotor(name='motor')
print(motor)

Now, we have a dummy motor with the default settings. All `Motor` objects have a bunch of methods and properties. The most important one is the methods to move the motor and to read its current position.

In [ ]:
import time
motor.move(2.)
time.sleep(2.)
motor.dial_position

As you can see, `move()` starts the motion, but it does not block the execution of the program. This is useful when moving many independent motors together. Of course, often you do want to wait for the motor to finish by blocking the calling code. This can be done by checking the `Motor`'s `busy()` method.

In [ ]:
motor.move(-2)
while motor.busy():
    time.sleep(.25)
    print(motor.dial_position)

That's really all we need to know about motors. But there are some convenient properties which make motor objects easier to user experimentally.

The first is that we can defined user positions while still keeping track of the positions reported by the hardware. This is useful for setting some motor to zero in a reference position, to change units, or to invert an axis. The transformation can set by changing the `user_position` attribute, which internally updates the `_offset` and `_scaling` variables. The positions are calculated as

$$\mathrm{(user\ position)} = \mathrm{(dial\ position)} * \mathrm{scaling} + \mathrm{offset}$$.

In [ ]:
print(motor.user_position, motor.dial_position)
motor.user_position = 12
print(motor.user_position, motor.dial_position, motor._offset)

For full control, you can write directly to the `_offset` and `_scaling` variables.

In [ ]:
print(motor.user_position)
motor._offset = 0
motor._scaling = -1
print(motor.user_position)

The second helpful property is that you can set software limits, which is a simple way to avoid making mistakes and crashing things at the beamline. We can set the limits using the library, either in dial or user coordinates. The `Motor` class automatically keeps track of dial and user limits by respecting the dial limits, re-calculating user limits as needed.

In [ ]:
motor._offset = 105
motor._scaling = 1
motor.user_limits = (90, 110)
print(motor.dial_limits)

At a beamline, these parameters are usually known, and can be declared when the motor object is created.

In [ ]:
motor2 = DummyMotor(name='motor2', scaling=-1, offset=100, dial_limits=(-5, 5))

### Motor macros

In practical beamline use, we often interact with motors using argument syntax macros instead of python syntax, just because macro syntax is more familiar to users and easier to type. The library module `contrast.motors` defines a bunch of macros to make motor handling more convenient. For example, we can print the current position of a motor using `%wm`.

In [ ]:
%wm motor2

As with the rest of contrast, macros are stored in the library together with the classes they are meant to operate on. So in the `contrast.motors.Motor` module, we can find familiar macros like `%wa`, `%umv`, `%umvr`, etc. After reading the above description of the `Motor` API, it should be quite clear to see how these macros operate on `Motor` objects, simply by calling `move()`, setting `_offset`, `_scaling`, and `dial_limits`.

### Creating our own `Motor`

To create your own `Motor` class, a subclass is created that handles peculiarities of whatever motor we want to interface. The `Motor` base class ([source code](https://contrast.readthedocs.io/en/latest/_modules/contrast/motors/Motor.html#Motor)) contains three methods which need writing. Two of these were introduced above, and the third is quite obvious.

1. `dial_position`, which reads the current motor position or initiates a movement,
2. `busy()`, which tells if the motor is doing something or whether it is at the commanded position, and
3. `stop()`, which stops any motion.

Here we'll create a dummy motor which simply goes directly to the commanded position.

In [ ]:
from contrast.motors import Motor

class PretendMotor(Motor):
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)  # call the base class constructor, Motor.__init__()
        self._pos = 0  # A variable for the commanded position
        
    @property
    def dial_position(self):
        # here we'd normally ask the physical motor where it is
        return self._pos

    @dial_position.setter
    def dial_position(self, pos):
        # here we'd normally tell the motor where to go
        self._pos = pos
        
    def busy(self):
        # here we'd normally ask the motor if it's on target
        return False

    def stop(self):
        # here we'd normally stop any motor motion
        print('stopping %s' % self.name)
        

First we set up the object and do any initialization. This is Python boilerplate and half the internet describes how to do it. Next we write the code that moves the motor and reads its position via `dial_position`. This is supplied as a python property, with a getter and a setter, and the second half of the internet explains how these work. Lastly, `busy` and `stop` are written, but in this case they won't do much since the motor goes directly to the target.

Now let's see if it works! Note that inheritaning `Motor` means that all macros, user positions and limits, work. Neat!

In [ ]:
pret = PretendMotor(name='pret', dial_limits=(-10, 10))
%mv pret 3.14
%wm pret

## Detectors

### Library basics

Detectors include actual X-ray detectors, but also any number or other piece of information that can be acquired from the beamline. It's often vert useful to be able to throw together a `Detector` subclass to interface with user equipment.

Detectors are also `Gadget`s, and as such need names. Let's make a dummy detector to see how it works.

In [ ]:
from contrast.detectors import DummyDetector
det = DummyDetector(name='det')

Now let's see how to operate on a detector. `contrast` defines a sequence of method calls for data acquisition.

1. `prepare()` sets up the detector for the coming measurements (scan),
2. `arm()` is called before every measurement (scan point),
3. `start()` is called to initiate every measurement (scan point),
4. `read()` is used after each measurement (scan point) to get the data.
5. `busy()` is used to see if a detector is busy measuring,
6. `stop()` is used to abort an ongoing measurement, and

A data acquisition routine is essentially just a loop calling these detector methods and perhaps moving motors and shuffling data in between. Each detector works differently, so what is acually done in `prepare()`, `arm()`, and `start()` differs. Some detectors (Eigers and Merlins) arm for hardware triggers already in `prepare()` and then just wait, some detectors need to be armed on every step, and some detectors simply grab a Tango value when you call `start()`.

Below is a simple data acquisition loop which reads 10 random values from our dummy motor.

In [ ]:
det.prepare(acqtime=.5, dataid=None)
for i in range(10):
    det.start()
    while det.busy():
        time.sleep(.1)
    print(det.read())

### Detector macros

Just like motors, a number of convenience macros are defined in the `contrast.detectors.Detector` module. For example, the `%lsdet` macro lists all current detectors.

In [ ]:
%lsdet

`Detector` objects have an `active` attribute which indicate whether that particular detector should be used data acquisition. The macros `%activate` and `%deactivate` toggle this flag in a helpful way. The `%lsdet` output also indicates whether each detector is active.

In [ ]:
det2 = DummyDetector(name='det2')
det3 = DummyDetector(name='det3')

%deactivate det
%deactivate det2
%lsdet

In [ ]:
%activate det*
%lsdet

### Writing our own detector

Writing `Detector` subclasses involves a few more methods than for 'Motor's. Depending on the specifics of the physical detector, many of these methods are essentially empty. Here, we'll write a detector which records a one-dimensional array of numbers at each measurement. For details on the arguments to `prepare()`, see the [base class documentation](file:///home/alex/sw/contrast/docs/build/html/contrast.detectors.html#contrast.detectors.Detector.Detector).

In [ ]:
from contrast.detectors import Detector
import numpy as np

class VectorDetector(Detector):

    def initialize(self):
        # normally, we'd set up connections to the detector here
        self._started = 0
        self._acqtime = 1.
    
    def prepare(self, acqtime, dataid=None, n_starts=None):
        self._acqtime = acqtime

    def arm(self):
        self.values = None  # clear any old data

    def start(self):
        self._started = time.time()
        self.values = (np.random.rand(10) * 256).astype(int)

    def stop(self):
        self._acqtime = 0.

    def busy(self):
        return (time.time() < self._started + self._acqtime)

    def read(self):
        return self.values

Now we can carry out the same data acquisition loop that we did above, using our brand new home-made dummy.

In [ ]:
vector = VectorDetector(name='vector')

vector.prepare(acqtime=.5, dataid=None)
for i in range(10):
    vector.start()
    while vector.busy():
        time.sleep(.1)
    print(vector.read())

In fact, this little loop represents most of what data acquisition is. We already have enough motors and detectors to use the full scanning machinery of `contrast`, which is basically just loops like this with prettier printing and passing of data to `Recorder` objects. We'll finish with running a simple contrast `dscan`.

In [ ]:
%dscan motor -2 2 50 .1